In [2]:
import numpy as np
import pandas as pd
from gensim.summarization.summarizer import summarize
from rouge import Rouge

## Summarization Without Classification

In [4]:
data = pd.read_csv("../data/news/2019-12-15_philippines_earthquake.csv")
data.head()

,source_name,title,url,date,content
0,Yahoo.com,Rescuers search building rubble after Philippi...,https://news.yahoo.com/rescuers-search-buildin...,2019-12-16,Rescuers pulled out two bodies Monday from a t...
1,Independent,Philippines earthquake: Powerful 6.9-magnitude...,https://www.independent.co.uk/news/world/asia/...,2019-12-15,One child reported to have died and at least 1...
2,Bostonglobe.com,Strong earthquake hits southern Philippines,https://www.bostonglobe.com/news/world/2019/12...,2019-12-15,An earthquake measuring magnitude 6.9 rocked t...
3,Dw.com,Major earthquake hits southern Philippines,https://www.dw.com/en/strong-earthquake-hits-s...,2019-12-15,A major earthquake hit the southern Philippine...
4,Yahoo.com,Child killed as quake strikes southern Philipp...,https://news.yahoo.com/strong-quake-hits-south...,2019-12-15,"Sunday's tremor cracked schools, toppled homes..."


In [19]:
text = " ".join(data.content.tolist())
text_clean = set(summarize(text, ratio=0.1).split("\n")) # remove duplicated sentences
print("\n".join(text_clean))

The magnitude 6.9 quake struck an area about 6 kilometers (3.7 miles) northwest of Padada town in Davao del Sur province at a depth of 30 kilometers (18 miles), according to the Philippine Institute of Volcanology and Seismology.
Police said a rescue operation had been launched at a heavily damaged market building in Padada near the 6.8 magnitude quake's epicentre, which is about 90 kilometres (55 miles) south of the major city of Davao.
A strong earthquake struck near the Philippine city of Davao on Sunday, killing a child, damaging buildings and causing several injuries, officials said, the latest in a series of tremors to strike the southern part of the country in recent months.
A powerful earthquake measuring 6.8-magnitude has struck the Philippines near the city of Davao, with one child reported to have died and 14 people been injured as tremors sent locals fleeing in panic from damaged homes and shopping centres.
Vincent Fernandez, the mayor of Matanao town in Davao del Sur provi

In [5]:
data.date.unique()

array(['2019-12-16', '2019-12-15'], dtype=object)

In [8]:
result = {}
for date in data.date.unique():
    text = " ".join(data[data.date == date].content.tolist())
    result[date] = summarize(text, ratio=0.1)

In [9]:
print(result["2019-12-15"])

A powerful earthquake measuring 6.8-magnitude has struck the Philippines near the city of Davao, with one child reported to have died and 14 people been injured as tremors sent locals fleeing in panic from damaged homes and shopping centres.
The quake was centred 38 miles southwest of Davao on the island of Mindanao, at a depth of 28.2km, the US Geological Survey said, revising down the magnitude from an earlier reported 6.9.
Vincent Fernandez, the mayor of Matanao town in Davao del Sur province, told Radio DZMM that a six-year-old girl had died after a wall collapsed on her.
One picture said to be from Padada, close to the epicentre in the province of Davao del Sur, showed a building collapsed onto a car and another people being rescued from beneath fallen rubble.
The Philippines sits on the geologically active Pacific “Ring of Fire” and experiences frequent tremors, including four strong quakes in October and November, which together killed at least 20 people.
Police said a rescue op

## Summarization with Classification

In [3]:
data = pd.read_csv("../data/Albania-news.csv")
data.head()

,Sentence,Section
0,Rama said on Saturday that preliminary figures...,building
1,"There have been more than 500 aftershocks, som...",building
2,"A 20-year-old woman, in a coma after she was i...",building
3,"In Durres, hundreds of residents as well as Ra...",building
4,One of the heavily damaged buildings has been ...,building


In [4]:
def get_summary(df, section, ratio=0.1):
    df = df[df.Section == section]
    text = " ".join(df.Sentence.tolist())
    text_clean = set(summarize(text, ratio=ratio).split("\n")) # remove duplicated sentences
    return " ".join(text_clean)

In [5]:
print(get_summary(data, "building", 0.2))

Four buildings, including a five-storey apartment block, collapsed in KodÃ«r-ThumanÃ« and the town was hardest hit from the earthquake. A 20-year-old woman, in a coma after she was injured by a falling brick while leaving her apartment in Tirana, died, the health ministry said on Saturday. â€œEverything was moving in an unbelievable rhythm, I could hear the walls cracking, dishes and glass breaking,â€  Videos and pictures shared on social media showed chaotic scenes of residents rummaging through the rubble, or trying to extricate people trapped under collapsed buildings. The earthquake struck at 3:54 a.m. near the Adriatic coast, about 19 miles west of Tirana, home to nearly 900,000 people. In Elbasan, a town about 35 miles from Durres, Olsi Shehi, a 39-year-old cook, said a four-story house had fallen, trapping people inside. About 2,500 people from damaged homes have been sheltered in hotels. Rescuers in Albania dug through the rubble of collapsed buildings in search of survivors o

In [6]:
print(get_summary(data, "resilience", 0.2))

The search-and-rescue operation for earthquake survivors in Albania has ended, with the death toll standing at 51 and no more bodies believed to be in the ruins, Prime Minister Edi Rama said. On 30 November Prime Minister Rama announced the end of the search and rescue operation, as no more bodies were expected to be under the rubble. In accordance with the Albanian constitution regarding an emergency situation, the Albanian parliament granted Prime Minister Edi Rama state of emergency powers to deal with earthquake aftermath. Prime Minister Rama said that the state budget was being reconfigured to manage the situation following the earthquake. In the immediate aftermath, 2,500 people became displaced by the earthquake and are temporarily being accommodated either in the Niko Dovana Stadium of DurrÃ«s in tents or in hotels. A state of emergency lasting 30 days was declared by the Albanian government for DurrÃ«s, ThumanÃ« and Tirana and later extended to LezhÃ« and LaÃ§. Blue and white 

In [7]:
print(get_summary(data, "infrastructure", 0.2))

In early February 2020, the Albanian government publicised figures that earthquake damage to private and public properties cost â‚¬844 million.


### Evaluation

In [8]:
briefing = pd.read_csv("../data/Albania-briefing.csv")
briefing.head()

,Text,Section
0,Albanian Prime Minister Edi Rama indicated tha...,Building
1,Because the earthquake caused significant buil...,Infrastructure
2,USGS PAGER tool estimated the fatalities to be...,Resilience


In [31]:
section = "resilience"
hyp = get_summary(data, section, 0.2)
ref = briefing[briefing.Section == "Resilience"].Text.values[0]

In [32]:
rouge = Rouge()
scores = rouge.get_scores(hyp, ref)

In [33]:
scores

[{'rouge-1': {'f': 0.3992869825356427,
   'p': 0.42105263157894735,
   'r': 0.37966101694915255},
  'rouge-2': {'f': 0.14669051379699902,
   'p': 0.15471698113207547,
   'r': 0.13945578231292516},
  'rouge-l': {'f': 0.3141025591077006,
   'p': 0.32450331125827814,
   'r': 0.30434782608695654}}]

In [ ]:
ref


'USGS PAGER tool estimated the fatalities to be between 1 and 10 with a probability of 12%, between 10 and 100 with a probability of 37%, between 100 and 1,000 with a probability of 37% and between 1,000 and 10,000 with a probability of 12% (Fig. 10). At the time this briefing was authored, the number of deaths as a consequence of the earthquake was reported as 51 and there were approximately 2,000 injuries. Damages were expected to be between $1 million and $10 million, between $10 million and $100 million, and between $100 million and $1,000 million with probabilities of 8%, 25% and 36%, respectively. Furthermore, there were probabilities of 22% and 6% of the economic loss to be between $1,000 million and $10,000 million and between $10,000 million and $100,000 million, respectively. Given the severity of the situation, Albanian Prime Minister Edi Rama declared a state of emergency in Tirana and Durres during December. Recovery efforts are currently continuing in the rubble of collap